#  Activeloop's Deep Lake

>[Activeloop's Deep Lake](https://docs.activeloop.ai/) as a Multi-Modal Vector Store that stores embeddings and their metadata including text, jsons, images, audio, video, and more. It saves the data locally, in your cloud, or on Activeloop storage. It performs hybrid search including embeddings and their attributes.

This notebook showcases basic functionality related to `Activeloop's Deep Lake`. While `Deep Lake` can store embeddings, it is capable of storing any type of data. It is a serverless data lake with version control, query engine and streaming dataloaders to deep learning frameworks.  

For more information, please see the Deep Lake [documentation](https://docs.activeloop.ai) or [api reference](https://docs.deeplake.ai)

In [ ]:
%pip install -qU openai "deeplake[all]" tiktoken

In [2]:
from open.text.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import DeepLake

In [7]:
from dotenv import load_dotenv
import os
load_dotenv()
query = "What did the president say about Ketanji Brown Jackson"
embeddings = OpenAIEmbeddings(openai_api_base=os.environ["EMBEDDINGS_API_BASE"])

In [3]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Create a dataset locally at `./deeplake/`, then run similarity search. The Deeplake+LangChain integration uses Deep Lake datasets under the hood, so `dataset` and `vector store` are used interchangeably. To create a dataset in your own cloud, or in the Deep Lake storage, [adjust the path accordingly](https://docs.activeloop.ai/storage-and-credentials/storage-options).

In [7]:
db = DeepLake(
    dataset_path="s3://langchain-deeplake/test", 
    creds="ENV", 
    embedding=embeddings, 
    overwrite=True
)
db.add_documents(docs)
# or shorter
# db = DeepLake.from_documents(docs, dataset_path="./my_deeplake/", embedding=embeddings, overwrite=True)
docs = db.similarity_search(query)

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Dataset(path='s3://langchain-deeplake/test', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (88, 1024)  float32   None   
    id        text      (88, 1)      str     None   
 metadata     json      (88, 1)      str     None   
   text       text      (88, 1)      str     None   


In [8]:
print(docs[0].page_content)

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


Later, you can reload the dataset without recomputing embeddings

In [8]:
db = DeepLake(
     dataset_path="s3://langchain-deeplake/test", creds="ENV", embedding=embeddings, read_only=True
)
docs = db.similarity_search(query)

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Deep Lake Dataset in s3://langchain-deeplake/test already exists, loading from the storage


In [9]:
print(docs[0].page_content)

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


Deep Lake, for now, is single writer and multiple reader. Setting `read_only=True` helps to avoid acquiring the writer lock.

### TQL Search

Furthermore, the execution of queries is also supported within the similarity_search method, whereby the query can be specified utilizing Deep Lake's Tensor Query Language (TQL).

In [10]:
search_id = db.vectorstore.dataset.id[0].numpy()

In [11]:
docs = db.similarity_search(
    query=None,
    tql_query=f"SELECT * WHERE id == '{search_id[0]}'",
)

In [12]:
docs

[Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny.', metadata={'source': './state_of_the_union.txt'}),
 Document(page_content='Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, thei

## Deep Lake API
you can access the Deep Lake  dataset at `db.vectorstore`

In [13]:
# get structure of the dataset
db.vectorstore.summary()

Dataset(path='s3://langchain-deeplake/test', read_only=True, tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (88, 1024)  float32   None   
    id        text      (88, 1)      str     None   
 metadata     json      (88, 1)      str     None   
   text       text      (88, 1)      str     None   


In [14]:
# get embeddings numpy array
embeds = db.vectorstore.dataset.embedding.numpy()

### Transfer local dataset to cloud
Copy already created dataset to the cloud. You can also transfer from cloud to local.

In [16]:
import deeplake

source = "s3://langchain-deeplake/test"  # could be local, s3, gcs, etc.
destination = "./langchain-deeplake/test"  # could be local, s3, gcs, etc.

deeplake.deepcopy(src=source, dest=destination, overwrite=True)

Copying dataset: 100%|██████████| 54/54 [00:18<00:00


Dataset(path='./langchain-deeplake/test', tensors=['embedding', 'id', 'metadata', 'text'])

In [17]:
db = DeepLake(dataset_path=destination, embedding=embeddings)
db.add_documents(docs)

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Deep Lake Dataset in ./langchain-deeplake/test already exists, loading from the storage


Dataset(path='./langchain-deeplake/test', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype       shape      dtype  compression
  -------    -------     -------    -------  ------- 
 embedding  embedding  (176, 1024)  float32   None   
    id        text      (176, 1)      str     None   
 metadata     json      (176, 1)      str     None   
   text       text      (176, 1)      str     None   


['dc7ab23a-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab23b-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab23c-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab23d-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab23e-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab23f-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab240-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab241-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab242-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab243-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab244-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab245-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab246-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab247-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab248-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab249-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab24a-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab24b-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab24c-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab24d-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab24e-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab24f-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab250-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab251-3bf9-11ee-ae7d-17ad7064eb8f',
 'dc7ab252-3bf9-